In [1]:
from collections import defaultdict
import json
import numpy as np
import os
import pandas as pd
import csv
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier

In [2]:
DATA_HOME = 'data'

REVIEWDATA_HOME = os.path.join(DATA_HOME, 'amazon_reviews_us_Camera_v1_00.tsv')

test = os.path.join(DATA_HOME, 'cs224u-colors-bakeoff-data.csv')


# GLOVE_HOME = os.path.join(DATA_HOME, 'glove.6B')


In [3]:
START_SYMBOL = "<s>"
END_SYMBOL = "</s>"
UNK_SYMBOL = "$UNK"

In [4]:
df = pd.read_csv(REVIEWDATA_HOME, sep='\t',error_bad_lines=False, warn_bad_lines=False)


In [5]:
df.columns.tolist()

['marketplace',
 'customer_id',
 'review_id',
 'product_id',
 'product_parent',
 'product_title',
 'product_category',
 'star_rating',
 'helpful_votes',
 'total_votes',
 'vine',
 'verified_purchase',
 'review_headline',
 'review_body',
 'review_date']

In [6]:
# Randomly select a small portion of the entire dataset for model implementation to save computation time
small_df = df.sample(frac=0.01, random_state=1).reset_index(drop=True)
print(small_df.shape)
small_df.head(3)

(18008, 15)


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,12491786,R1C0EVYBV2Z18Z,B000U92DLA,845576079,Kastar Camera & Camcorder Battery Home Travel ...,Camera,5,6,6,N,Y,A Must For On The Go,"Purchased the charger from Nextop2100, arrived...",2008-11-22
1,US,37280009,R3Q837V78NS6B9,B0008GCYNW,615481480,Olympus Camedia C755 4MP Digital Camera with 1...,Camera,3,15,17,N,Y,The Good and the Not-so Good,I was attracted to this camera for three reaso...,2005-09-08
2,US,16532896,R22CZCJH5YNX1O,B003CZ7L84,884672402,Intova CP-9 Compact Digital Camera with 130 fe...,Camera,4,1,1,N,Y,Great for what we paid!,We took it with us to Cancun. My husband took ...,2011-11-02


In [7]:
# Select features that may by useful for model training
features = ['product_id', 'star_rating', 'helpful_votes', 'total_votes', 'verified_purchase', 'review_headline', 'review_body']
small_df_selectedFeatures = small_df[features]

# Convert 5-star ratings into 2 (positive and negative) or 3 (positive, neutral and negative) classes
def starRating_to_PosNegClass(dataset, num_class):
    if num_class == 2:    
        # Negative reviews: 1-3 Stars = 0 ; Positive reviews: 4-5 Stars = 1
        dataset['pos_neg'] = [1 if x > 3 else 0 for x in dataset.star_rating]
    if num_class == 3: 
        # Negative reviews: 1-2 Stars = -1 ; Positive reviews: 4-5 Stars = 1 ; Neutral Reviews: 3 Stars = 0
        dataset['pos_neg_1'] = [1 if x > 3 else 0 for x in dataset.star_rating] 
        dataset['pos_neg_2'] = [-1 if x < 3 else 0 for x in dataset.star_rating] 
        dataset['pos_neg'] = (dataset['pos_neg_1'] + dataset['pos_neg_2'])
        dataset.drop(['pos_neg_1', 'pos_neg_2'], axis=1)
    return dataset


small_df_selectedFeatures = starRating_to_PosNegClass(small_df_selectedFeatures, num_class=2)
print('# of positive reviews and negative reviews: ')
small_df_selectedFeatures['pos_neg'].value_counts()

# of positive reviews and negative reviews: 


C:\Users\Public\Miniconda3\envs\nlu\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


1    13925
0     4083
Name: pos_neg, dtype: int64

In [8]:
small_df_selectedFeatures.head(5)

,product_id,star_rating,helpful_votes,total_votes,verified_purchase,review_headline,review_body,pos_neg
0,B000U92DLA,5,6,6,Y,A Must For On The Go,"Purchased the charger from Nextop2100, arrived...",1
1,B0008GCYNW,3,15,17,Y,The Good and the Not-so Good,I was attracted to this camera for three reaso...,0
2,B003CZ7L84,4,1,1,Y,Great for what we paid!,We took it with us to Cancun. My husband took ...,1
3,B000EB7OTU,5,0,1,N,Great Bag,This bag is perfect for our nikon d80. It has...,1
4,B00MIB0JG4,1,0,0,Y,CRAP Don't waist the time and money,"Crapy cheap plastic, leg broke the second I to...",0


In [9]:
# CountVectorizer -> 應該是 bag of word? 不知道我有沒有理解錯誤

def word_counts_featurizer(data):
    model = CountVectorizer()  # Convert a collection of text documents to a matrix of token counts
    X = model.fit_transform(data)
    print('# features: {}'.format(X.shape[1]))
    return X

def KNN(X_train, Y_train, X_test, Y_test, n_neighbors):
    knn = KNeighborsClassifier(n_neighbors = n_neighbors)
    knn_model_1 = knn.fit(X_train, Y_train)
    print('k-NN accuracy for test set: %f' % knn_model_1.score(X_test, Y_test))

small_X = word_counts_featurizer(small_df_selectedFeatures['review_body'])
small_Y = small_df_selectedFeatures['pos_neg']
small_X_train, small_X_test, small_Y_train, small_Y_test = train_test_split(small_X, small_Y, test_size=0.3, random_state=0)
KNN(small_X_train, small_Y_train, small_X_test, small_Y_test, n_neighbors=5)

# features: 26179
k-NN accuracy for test set: 0.794188


In [13]:
# 從這個cell開始, 我主要是想要create a new column. 
# 這個新col 內容和 review_body 一樣, 只是經過一些處理, 例如remove punctuation, lower the case and then split

def text_processing(data):
    data['review_body'] = data['review_body'].astype(str)

    #lower the case and then split
    data['review_body_1'] = START_SYMBOL + ' ' + data['review_body'] + ' ' +  END_SYMBOL
    data['review_body_2'] = data['review_body_1'].str.lower().str.split(' ', expand = False) 
    data.drop(['review_body_1'], axis=1)
    return data

# Note: if we want to remove punctuation, can use: df['col_name'].str.replace('[^\w\s]','') 
small_df_selectedFeatures = text_processing(small_df_selectedFeatures)
small_df_selectedFeatures.head(3)

C:\Users\Public\Miniconda3\envs\nlu\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\Public\Miniconda3\envs\nlu\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Public\Miniconda3\envs\nlu\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

,product_id,star_rating,helpful_votes,total_votes,verified_purchase,review_headline,review_body,pos_neg,review_body_1,review_body_2
0,B000U92DLA,5,6,6,Y,A Must For On The Go,"Purchased the charger from Nextop2100, arrived...",1,"<s> Purchased the charger from Nextop2100, arr...","[<s>, purchased, the, charger, from, nextop210..."
1,B0008GCYNW,3,15,17,Y,The Good and the Not-so Good,I was attracted to this camera for three reaso...,0,<s> I was attracted to this camera for three r...,"[<s>, i, was, attracted, to, this, camera, for..."
2,B003CZ7L84,4,1,1,Y,Great for what we paid!,We took it with us to Cancun. My husband took ...,1,<s> We took it with us to Cancun. My husband t...,"[<s>, we, took, it, with, us, to, cancun., my,..."


In [15]:
# train, test = train_test_split(df, test_size=0.3)
# print('training dataset: ', train.shape)
# print('test dataset: ', test.shape)

small_train, small_test = train_test_split(small_df_selectedFeatures, test_size=0.3, random_state=0)
print('small training dataset: ', small_train.shape)
print('smalle test dataset: ', small_test.shape)


small training dataset:  (12605, 10)
smalle test dataset:  (5403, 10)


In [16]:
# 這邊是仿造 HW4 Q1 部分, 不過還沒研究怎麼套用到老師寫的model裡

small_train_text = small_train['review_body_2'].values.tolist()
small_train_vocab = sorted(set([item for sublist in small_train_text for item in sublist])) + [UNK_SYMBOL]

print(len(small_train_vocab))


52442
